In [ ]:
# Import Libraries

In [ ]:
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

# Preprocessing & modeling
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, brier_score_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, IsolationForest
import xgboost as xgb
import lightgbm as lgb
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import joblib

# Unsupervised & explainability
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
import umap
import shap

# Deep learning autoencoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Paths
PROJECT_DIR = Path('.')
DATA_DIR = PROJECT_DIR / 'data'
MODEL_DIR = PROJECT_DIR / 'models' / 'save'
DATA_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)
print('Model dir:', MODEL_DIR.resolve())

In [ ]:
# Load dataset

In [ ]:
possible_files = ['framingham.csv', 'Framingham.csv', 'framingham_heart.csv', 'framingham_heart_study.csv']
for f in possible_files:
p = DATA_DIR / f
if p.exists():
df = pd.read_csv(p)
print(f'Loaded {p.name} — shape:', df.shape)
break
else:
raise FileNotFoundError(f"No framingham csv found in {DATA_DIR}. Place the dataset file there with one of names: {possible_files}")

# quick overview
print(df.columns.tolist())
df.head()

In [ ]:
print('Shape:', df.shape)
print('
Missing values per column:')
print(df.isnull().sum())
print('
Target distribution:')
print(df['TenYearCHD'].value_counts())

In [ ]:
# Feature engineering function

In [ ]:
def engineer_features(df_in):
df = df_in.copy()
# pulse pressure
if 'sysBP' in df.columns and 'diaBP' in df.columns:
df['pulse_pressure'] = df['sysBP'] - df['diaBP']
# BMI category
if 'BMI' in df.columns:
df['bmi_cat'] = pd.cut(df['BMI'], bins=[0,18.5,25,30,200], labels=['underweight','normal','overweight','obese'])
# age decade
if 'age' in df.columns:
df['age_decade'] = (df['age']//10)*10
# pack proxy
if 'cigsPerDay' in df.columns:
df['pack_proxy'] = df['cigsPerDay'] * df.get('smokingYears', 1)
# Cast categories
for c in ['bmi_cat','age_decade']:
if c in df.columns:
df[c] = df[c].astype('category')
return df